In [1]:
# import pickle as p
import os

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.ops import standard_ops
%matplotlib inline

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/",one_hot=False)

def strip_first_col(fname, delimiter=None):
    with open(fname, 'rb') as fin:
        for line in fin:
            try:
               yield line.split(delimiter, 1)[1]
            except IndexError:
               continue


In [3]:
train_data = np.load("./train10_data.npy")
train_label = np.load("./train10_label.npy")
test_data = np.load("./test10_data.npy")
test_label = np.load("./test10_label.npy")

## autoencoder pretrain

In [4]:
def l1_regularizer(scale, x):
    return standard_ops.multiply(scale, standard_ops.reduce_sum(standard_ops.abs(x)))

In [5]:
def batchnorm(Ylogits, is_test, iteration, offset, convolutional=False):
    exp_moving_avg = tf.train.ExponentialMovingAverage(0.999, iteration) # adding the iteration prevents from averaging across non-existing iterations
    bnepsilon = 1e-5
    if convolutional:
        mean, variance = tf.nn.moments(Ylogits, [0, 1, 2])
    else:
        mean, variance = tf.nn.moments(Ylogits, [0])
    update_moving_everages = exp_moving_avg.apply([mean, variance])
    m = tf.cond(is_test, lambda: exp_moving_avg.average(mean), lambda: mean)
    v = tf.cond(is_test, lambda: exp_moving_avg.average(variance), lambda: variance)
    Ybn = tf.nn.batch_normalization(Ylogits, m, v, offset, None, bnepsilon)
    return Ybn, update_moving_everages

def no_batchnorm(Ylogits, is_test, iteration, offset, convolutional=False):
    return Ylogits, tf.no_op()


In [6]:
CAPACITY = 30000
training_epochs = 10
display_step = 1
examples_to_show = 10
n_input = 22283 #Need to change to 22283
l1_scale = 0.001
l2_scale = 0.001

# tf Graph input (only pictures)
X = tf.placeholder("float",[None,n_input])
learning_rate = tf.placeholder(tf.float32, shape=[])
# X = get_batch(data,BATCH_SIZE,CAPACITY)
tst = tf.placeholder(tf.bool)
# training iteration
iter = tf.placeholder(tf.int32)
keep_prob = tf.placeholder(tf.float32)  

#need to change to 4 layers 
n_hidden_1 = 200
n_hidden_2 = 150
n_hidden_3 = 100
n_hidden_4 = 42

weights= {
    'encoder_h1': tf.Variable(tf.truncated_normal([n_input,n_hidden_1], stddev=0.1)),
    'encoder_h2': tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2], stddev=0.1)),
    'encoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3], stddev=0.1)),
    'encoder_h4': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4], stddev=0.1)),
    'decoder_h1': tf.Variable(tf.truncated_normal([n_hidden_4,n_hidden_3], stddev=0.1)),
    'decoder_h2': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_2], stddev=0.1)),
    'decoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_1], stddev=0.1)),
    'decoder_h4': tf.Variable(tf.truncated_normal([n_hidden_1,n_input], stddev=0.1)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'encoder_b4': tf.Variable(tf.random_normal([n_hidden_4])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_3])),
    'decoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b3': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b4': tf.Variable(tf.random_normal([n_input])),
}
# weightse1_summary = tf.summary.histogram("weights_e1", weights['encoder_h1'])
# weightse2_summary = tf.summary.histogram("weights_e2", weights['encoder_h2'])
# weightse3_summary = tf.summary.histogram("weights_e3", weights['encoder_h3'])
# weightsd1_summary = tf.summary.histogram("weights_d1", weights['decoder_h1'])
# weightsd2_summary = tf.summary.histogram("weights_d2", weights['decoder_h1'])
# weightsd3_summary = tf.summary.histogram("weights_d3", weights['decoder_h1'])

update_ema = []
update_emA = []
def encoder(x):  
    Y1l = tf.add(tf.matmul(x, weights['encoder_h1']),biases['encoder_b1'])
    Y1bn, update_ema1 = batchnorm(Y1l, tst, iter, biases['encoder_b1'])
    layer_1 = tf.nn.relu(tf.nn.dropout(Y1bn, keep_prob))
    
    Y2l = tf.add(tf.matmul(layer_1, weights['encoder_h2']),biases['encoder_b2'])
    Y2bn, update_ema2 = batchnorm(Y2l, tst, iter, biases['encoder_b2'])
    layer_2 = tf.nn.relu(tf.nn.dropout(Y2bn, keep_prob))
    
    Y3l = tf.add(tf.matmul(layer_2, weights['encoder_h3']),biases['encoder_b3'])
    Y3bn, update_ema3 = batchnorm(Y3l, tst, iter, biases['encoder_b3'])
    layer_3 = tf.nn.relu(tf.nn.dropout(Y3bn, keep_prob))
    
    Y4l = tf.add(tf.matmul(layer_3, weights['encoder_h4']),biases['encoder_b4'])
    Y4bn, update_ema4 = batchnorm(Y4l, tst, iter, biases['encoder_b4'])
    layer_4 = tf.nn.relu(Y4bn)
    
    global update_ema
    update_emA = tf.group(update_ema1, update_ema2, update_ema3, update_ema4)
    return layer_4
  
def decoder(x):  
#     layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['decoder_h1']),  biases['decoder_b1']))  
#     layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2']))  
#     layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['decoder_h3']),biases['decoder_b3']))
#     layer_4 = tf.nn.relu(tf.add(tf.matmul(layer_3, weights['decoder_h4']),biases['decoder_b4']))
    Y1l = tf.add(tf.matmul(x, weights['decoder_h1']),biases['decoder_b1'])
    Y1bn, update_ema5 = batchnorm(Y1l, tst, iter, biases['decoder_b1'])
    layer_1 = tf.nn.relu(tf.nn.dropout(Y1bn, keep_prob))
    
    Y2l = tf.add(tf.matmul(layer_1, weights['decoder_h2']),biases['decoder_b2'])
    Y2bn, update_ema6 = batchnorm(Y2l, tst, iter, biases['decoder_b2'])
    layer_2 = tf.nn.relu(tf.nn.dropout(Y2bn, keep_prob))

    Y3l = tf.add(tf.matmul(layer_2, weights['decoder_h3']),biases['decoder_b3'])
    Y3bn, update_ema7 = batchnorm(Y3l, tst, iter, biases['decoder_b3'])
    layer_3 = tf.nn.relu(tf.nn.dropout(Y3bn, keep_prob))
    
    Y4l = tf.add(tf.matmul(layer_3, weights['decoder_h4']),biases['decoder_b4'])
    Y4bn, update_ema8 = batchnorm(Y4l, tst, iter, biases['decoder_b4'])
    layer_4 = tf.nn.relu(Y4bn)
    
    global update_ema
    update_ema = tf.group(update_ema5, update_ema6, update_ema7, update_ema8)
    return layer_4



encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

y_pred = decoder_op
y_true = X

In [7]:
cost = tf.reduce_mean(tf.pow(y_true-y_pred,2))\
    + l2_scale * (tf.nn.l2_loss(weights['encoder_h1'])+tf.nn.l2_loss(biases['encoder_b1']))\
    + (l1_regularizer(l1_scale, weights['encoder_h4'])+l1_regularizer(l1_scale, biases['encoder_b4']))\
    + (l1_regularizer(l1_scale, weights['decoder_h1'])+l1_regularizer(l1_scale, biases['decoder_b1']))
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
cost_summary = tf.summary.scalar('Cost', cost)  
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

sess = tf.Session()
init = tf.global_variables_initializer()  
sess.run(init)  

In [8]:
index = np.random.permutation(train_data.shape[0])

def get_batch_np(itert, batchsize):
    batch_index = index[itert*batchsize:(itert+1)*batchsize]
    batch_data = train_data[batch_index]
    return batch_data


total_step = 0

In [10]:
BATCH_SIZE = 500
training_epochs = 10
# 首先计算总批数，保证每次循环训练集中的每个样本都参与训练，不同于批量训练  
total_batch = int(train_data.shape[0]/BATCH_SIZE) #总批数  

merged_summary_op = tf.summary.merge([cost_summary])
summary_writer = tf.summary.FileWriter('./tmp10/logs', sess.graph)

for epoch in range(training_epochs):  
    index = np.random.permutation(train_data.shape[0])
    for i in range(total_batch):  
        total_step += 1
        batch_xs= get_batch_np(i, BATCH_SIZE)  # max(x) = 1, min(x) = 0  
        
        # Run optimization op (backprop) and cost op (to get loss value)  
        summary_str, _, c = sess.run([merged_summary_op, optimizer, cost], feed_dict={X: batch_xs, learning_rate:1, tst: False, keep_prob:0.5})
        sess.run([update_ema, update_emA], {X: batch_xs, tst: False, iter: total_step, keep_prob:0.5})
        
        summary_writer.add_summary(summary_str, total_step)
        summary_writer.add_graph(sess.graph)
        
    if epoch % display_step == 0:  
        record = open("train10_loss.txt", "a+")
        record.write("epoch:"+str(epoch)+" loss:"+"{:.9f}\n".format(c))
        record.close()
        print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c)  
#         init_op = tf.global_variables_initializer()
#         saver = tf.train.Saver()
#         with tf.Session() as sesstemp:
#             sesstemp.run(init_op)
#             saver_path = saver.save(sesstemp, "model.ckpt")
#             print "Model saved in file: ", saver_path
print("Optimization Finished!")  

Epoch: 0001 cost= 85.734306335
Epoch: 0002 cost= 42.165725708
Epoch: 0003 cost= 18.135400772
Epoch: 0004 cost= 8.758343697
Epoch: 0005 cost= 5.010951042
Epoch: 0006 cost= 3.467998743
Epoch: 0007 cost= 2.875286579
Epoch: 0008 cost= 2.646712065
Epoch: 0009 cost= 2.541959763
Epoch: 0010 cost= 2.530435085
Optimization Finished!


## classify 

In [11]:
n_class = 10

y = tf.placeholder("float", shape=[None, 10]) 
classify = tf.Variable(tf.truncated_normal([n_hidden_4, n_class], stddev=0.1), tf.float32, name='cf')
weightse1_summary = tf.summary.histogram("weights_e1", weights['encoder_h1'])
weightse2_summary = tf.summary.histogram("weights_e2", weights['encoder_h2'])
weightse3_summary = tf.summary.histogram("weights_e3", weights['encoder_h3'])
weightse4_summary = tf.summary.histogram("weights_e4", weights['encoder_h4'])
cf_summary = tf.summary.histogram("weights_cf", classify)

def binaryclassify(x):
    out = tf.matmul(x, classify)
    return out

# encoder_op = encoder(X)
py = binaryclassify(encoder_op)
# cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(py), reduction_indices=[1]))
# cross_entropy = -tf.reduce_mean(y*tf.log(tf.clip_by_value(py,1e-10,1.0)))
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=py))\
            + l2_scale * (tf.nn.l2_loss(weights['encoder_h1'])+tf.nn.l2_loss(biases['encoder_b1']))\
            + (l1_regularizer(l1_scale, weights['encoder_h4'])+l1_regularizer(l1_scale, biases['encoder_b4']))\
            + l1_regularizer(l1_scale, classify) 
# decoder_op = decoder(encoder_op)

# cost = tf.reduce_mean(tf.pow(y_true-y_pred,2))
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
cost_summary = tf.summary.scalar('cross_entropy', cross_entropy)  
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam').minimize(cross_entropy)


correct_prediction = tf.equal(tf.argmax(py,1), tf.argmax(y, 1))
trainacc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
trainacc_summary = tf.summary.scalar('trainacc', trainacc) 

testacc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
testacc_summary = tf.summary.scalar('testacc', testacc) 

In [8]:
# CAPACITY = 30000
# training_epochs = 10
# learning_rate = tf.placeholder(tf.float32, shape=[])
# display_step = 1
# examples_to_show = 10
# n_input = 22283 #Need to change to 22283
# l1_scale = 0.001
# l2_scale = 0.001

# # tf Graph input (only pictures)

# X = tf.placeholder("float",[None,n_input])
# y = tf.placeholder("float", shape=[None, 2])  
# # X = get_batch(data,BATCH_SIZE,CAPACITY)

# #need to change to 4 layers 
# n_hidden_1 = 200
# n_hidden_2 = 150
# n_hidden_3 = 100
# n_hidden_4 = 42
# n_class = 2

# weights= {
#     'encoder_h1': tf.Variable(tf.truncated_normal([n_input,n_hidden_1], stddev=0.1), name='encoder_h1'),
#     'encoder_h2': tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2], stddev=0.1), name='encoder_h2'),
#     'encoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3], stddev=0.1), name='encoder_h3'),
#     'encoder_h4': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4], stddev=0.1), name='encoder_h4'),
#     'decoder_h1': tf.Variable(tf.truncated_normal([n_hidden_4,n_hidden_3], stddev=0.1), name='decoder_h1'),
#     'decoder_h2': tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_2], stddev=0.1), name='decoder_h2'),
#     'decoder_h3': tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_1], stddev=0.1), name='decoder_h3'),
#     'decoder_h4': tf.Variable(tf.truncated_normal([n_hidden_1,n_input], stddev=0.1), name='decoder_h4'),
# }


# classify = tf.Variable(tf.truncated_normal([n_hidden_4, n_class], stddev=0.1), tf.float32, name='cf')
# biases = {
#     'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1]), name='encoder_b1'),
#     'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2]), name='encoder_b2'),
#     'encoder_b3': tf.Variable(tf.random_normal([n_hidden_3]), name='encoder_b3'),
#     'encoder_b4': tf.Variable(tf.random_normal([n_hidden_4]), name='encoder_b4'),
#     'decoder_b1': tf.Variable(tf.random_normal([n_hidden_3]), name='decoder_b1'),
#     'decoder_b2': tf.Variable(tf.random_normal([n_hidden_2]), name='decoder_b2'),
#     'decoder_b3': tf.Variable(tf.random_normal([n_hidden_1]), name='decoder_b3'),
#     'decoder_b4': tf.Variable(tf.random_normal([n_input]), name='decoder_b4'),
# }

# weightse1_summary = tf.summary.histogram("weights_e1", weights['encoder_h1'])
# weightse2_summary = tf.summary.histogram("weights_e2", weights['encoder_h2'])
# weightse3_summary = tf.summary.histogram("weights_e3", weights['encoder_h3'])
# weightse4_summary = tf.summary.histogram("weights_e4", weights['encoder_h4'])
# cf_summary = tf.summary.histogram("weights_cf", classify)

# # def encoder(x):  
# layer_1 = tf.nn.relu(tf.add(tf.matmul(X, weights['encoder_h1']), biases['encoder_b1']))  
# layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))  
# layer_3 = tf.nn.relu(tf.add(tf.matmul(layer_2, weights['encoder_h3']), biases['encoder_b3']))
# layer_4 = tf.nn.relu(tf.add(tf.matmul(layer_3, weights['encoder_h4']), biases['encoder_b4']))
# #     return layer_4
# #encoder_op = layer_4

# def decoder(x):  
#     layer_1 = tf.nn.tanh(tf.add(tf.matmul(x, weights['decoder_h4']),  
#                                    biases['decoder_b4']))  
# #     layer_2 = tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['decoder_h2']),  
# #                                    biases['decoder_b2']))  
# #     layer_3 = tf.nn.tanh(tf.add(tf.matmul(layer_2, weights['decoder_h3']),
# #                                    biases['decoder_b3']))
# #     layer_4 = tf.nn.tanh(tf.add(tf.matmul(layer_3, weights['decoder_h4']),
# #                                    biases['decoder_b4']))
#     return layer_1

# def binaryclassify(x):
#     out = tf.matmul(x, classify)
#     return out

# # encoder_op = encoder(X)
# py = binaryclassify(encoder_op)
# # cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(py), reduction_indices=[1]))
# # cross_entropy = -tf.reduce_mean(y*tf.log(tf.clip_by_value(py,1e-10,1.0)))
# cross_entropy = tf.reduce_mean(
#     tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=py))\
#             + l2_scale * (tf.nn.l2_loss(weights['encoder_h1'])+tf.nn.l2_loss(biases['encoder_b1']))\
#             + (l1_regularizer(l1_scale, weights['encoder_h4'])+l1_regularizer(l1_scale, biases['encoder_b4']))\
#             + l1_regularizer(l1_scale, classify)       
# # decoder_op = decoder(encoder_op)

# # y_pred = predict
# # y_true = y

# # cost = tf.reduce_mean(tf.pow(y_true-y_pred,2))
# # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true))
# cost_summary = tf.summary.scalar('cross_entropy', cross_entropy)  
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, name='Adam').minimize(cross_entropy)


# correct_prediction = tf.equal(tf.argmax(py,1), tf.argmax(y, 1))
# trainacc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# trainacc_summary = tf.summary.scalar('trainacc', trainacc) 

# testacc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# testacc_summary = tf.summary.scalar('testacc', testacc) 

In [12]:
init = tf.global_variables_initializer() 
saver = tf.train.Saver()
sess = tf.Session()
new = 1
total_step = 0
if new == 1:
    sess.run(init)
else:
    init = tf.global_variables_initializer() 
    sess.run(init)
    saver.restore(sess, 'model10.ckpt')
    total_step = np.load("./total10_step.npy")

In [15]:
# saver = tf.train.Saver()
# sess = tf.Session()
# # new_saver = tf.train.import_meta_graph('model.ckpt.meta')
# init = tf.global_variables_initializer() 
# sess.run(init)
# saver.restore(sess, 'model.ckpt')

In [16]:
# all_vars = tf.trainable_variables()
# for v in all_vars:
#     print v.name
# sess.run("cf:0")
# #     sess.run(classify)

In [13]:
## change test set to one-hot
test_index = np.random.permutation(test_data.shape[0])
tl_list = test_label[test_index]
tl = np.zeros((test_label.shape[0], 10), dtype=np.int)
line = 0
td = test_data[test_index]
for i in tl_list:
    tl[line][i] = 1
    line += 1

In [14]:
index = np.random.permutation(train_data.shape[0])
def get_batch_np(itert, batchsize):
    batch_index = index[itert*batchsize:(itert+1)*batchsize]
    batch_data = train_data[batch_index]
    label_list = train_label[batch_index]
    batch_label = np.zeros((batchsize, 10), dtype=np.int)
    line = 0
    for i in label_list:
        batch_label[line][i] = 1
        line += 1
    return batch_data, batch_label

In [ ]:
BATCH_SIZE = 500
training_epochs = 100
test_display_step = 10
# 首先计算总批数，保证每次循环训练集中的每个样本都参与训练，不同于批量训练  
total_batch = int(train_data.shape[0]/BATCH_SIZE) #总批数  
 
merged_summary_op = tf.summary.merge([cost_summary, weightse1_summary, weightse2_summary, weightse3_summary, weightse4_summary,
                                     trainacc_summary, cf_summary])
summary_writer = tf.summary.FileWriter('./tmp10/logs', sess.graph)
summary_writer.add_graph(sess.graph)


for epoch in range(training_epochs):  
    index = np.random.permutation(train_data.shape[0])
    for i in range(total_batch):  
        total_step += 1
        batch_xs, batch_ys= get_batch_np(i, BATCH_SIZE)  # max(x) = 1, min(x) = 0  
        # Run optimization op (backprop) and cost op (to get loss value)
        
       # _, c = sess.run([optimizer, cross_entropy], feed_dict={X: batch_xs, y: batch_ys, learning_rate:0.0001})
        
        summary_str, _, c, ta = sess.run([merged_summary_op, optimizer, cross_entropy, trainacc], 
                                               feed_dict={X: batch_xs, y: batch_ys, learning_rate:0.0001, tst: False, keep_prob:0.5})
        sess.run([update_ema, update_emA], {X: batch_xs, y: batch_ys, tst: False, iter: total_step, keep_prob:0.5})
        summary_writer.add_summary(summary_str, total_step)
        
    if epoch % display_step == 0:  
        record = open("train10_loss.txt", "a+")
        record.write("epoch:"+str(epoch)+" loss:"+"{:.9f}\n".format(c))
        record.close()
        print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c) , "trainacc = ", "{:.3f}".format(ta)
    if epoch % test_display_step == 0:
        summary_str, tsta = sess.run([tf.summary.merge([testacc_summary]), testacc], feed_dict={X: td, y: tl, tst: False, keep_prob:0.5})
        summary_writer.add_summary(summary_str, total_step)
        print "TestAcc = ", "{:.3f}".format(tsta)
#         correct_prediction = tf.equal(tf.argmax(py,1), tf.cast(y, tf.int64))
        
#         accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#         batch_xs, batch_ys= get_batch_np(0, 5000)
#         cross_entropy = -tf.reduce_mean(y*tf.log(py))
        # cross_entropy2 = -tf.reduce_mean(tf.cast(y, tf.int64)*tf.log(py))
#         print(sess.run([accuracy, cross_entropy], feed_dict={X: test_data, y: test_label.reshape(1180,1)}))

# sess.run(tf.global_variables_initializer())
# saver = tf.train.Saver()
saver_path = saver.save(sess, "model10.ckpt")
print "Model saved in file: ", saver_path
np.save("total10_step", total_step)
print("Optimization Finished!")  

Epoch: 0001 cost= 19.795564651 trainacc =  0.068
TestAcc =  0.045
Epoch: 0002 cost= 19.526573181 trainacc =  0.076
Epoch: 0003 cost= 19.220760345 trainacc =  0.094
Epoch: 0004 cost= 18.981128693 trainacc =  0.094
Epoch: 0005 cost= 18.696603775 trainacc =  0.136
Epoch: 0006 cost= 18.446628571 trainacc =  0.152
Epoch: 0007 cost= 18.179264069 trainacc =  0.166
Epoch: 0008 cost= 17.910600662 trainacc =  0.200
Epoch: 0009 cost= 17.674171448 trainacc =  0.204
Epoch: 0010 cost= 17.466806412 trainacc =  0.204
Epoch: 0011 cost= 17.193330765 trainacc =  0.258
TestAcc =  0.154
Epoch: 0012 cost= 16.918409348 trainacc =  0.310
Epoch: 0013 cost= 16.715749741 trainacc =  0.306
Epoch: 0014 cost= 16.465320587 trainacc =  0.318
Epoch: 0015 cost= 16.264657974 trainacc =  0.334
Epoch: 0016 cost= 16.005458832 trainacc =  0.352
Epoch: 0017 cost= 15.797198296 trainacc =  0.346
Epoch: 0018 cost= 15.516515732 trainacc =  0.418
Epoch: 0019 cost= 15.358511925 trainacc =  0.376
Epoch: 0020 cost= 15.163107872 trai

In [ ]:
3+3

* Training set acc

In [20]:
correct_prediction = tf.equal(tf.argmax(py,1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
batch_xs, batch_ys= get_batch_np(0, 3000)
# cross_entropy2 = -tf.reduce_mean(tf.cast(y, tf.int64)*tf.log(py))
print(sess.run([accuracy], feed_dict={X: batch_xs, y: batch_ys}))

[0.98533332]


In [10]:
[0.96866667]

[0.96866667]

* Test set acc

In [28]:
correct_prediction = tf.equal(tf.argmax(py,1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# batch_xs, batch_ys= get_batch_np(0, 4000)
# cross_entropy2 = -tf.reduce_mean(tf.cast(y, tf.int64)*tf.log(py))
print(sess.run([accuracy], feed_dict={X: td, y: tl, tst: False, keep_prob:1}))

[0.92966104]


In [ ]:
[0.93220341]

In [28]:
#download teh MNIST data in folder "MNIST_data" that in the same path as this *.py
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

import tensorflow as tf

#图片的占位
x = tf.placeholder(tf.float32, [None, 784])

#系数
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

#softmax层
y = tf.nn.softmax(tf.matmul(x, W) + b)

#用于训练的真实值占位
y_ = tf.placeholder(tf.float32, [None, 10])

#交叉熵：-tf.reduce_sum(y_ * tf.log(y)是一个样本的，外面的tf.reduce_mean是batch的
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

#规定训练的方法：注意：使用GradientDescentOptimizer适合上述的误差项
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#初始化
init = tf.initialize_all_variables()

sess = tf.Session()
sess.run(init)

#训练
for i in range(5):
  batch_xs, batch_ys = mnist.train.next_batch(2)
  #print batch_xs.shape
  a,_ = sess.run([y, train_step], feed_dict={x: batch_xs, y_: batch_ys})
  print a

#验证，argmax(y,1)是获得y的第一个维度（即每一行）的最大值的位置
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run([accuracy,tf.shape(y)], feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
[[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]]
[[  1.74958725e-02   1.74958725e-02   1.74958725e-02   1.74958725e-02
    1.74958725e-02   1.74958725e-02   1.74958725e-02   5.23108721e-01
    1.74958725e-02   3.36924314e-01]
 [  1.90278515e-04   1.90278515e-04   1.90278515e-04   1.90278515e-04
    1.90278515e-04   1.90278515e-04   1.90278515e-04   1.68166868e-02
    1.90278515e-04   9.81661081e-01]]
[[  1.77685414e-02   5.68983814e-05   5.68983814e-05   5.68983814e-05
    5.68983814e-05   5.68983814e-05   5.68983814e-05   8.84178877e-01
    9.21878219e-02   5.52334916e-03]
 [  6.64212108e-02   6.22349384e-04   6.22349384e-04   6.22349384e-04
    6.22349384e-04   6.

In [16]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=False)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [18]:
mnist.test.labels.shape

(10000,)